# **Composite Strength Prediction Function**
### Version 1 | Amil Khan, Marat I. Latypov
***

### **Import Packages**

`PyMKS`
- The Materials Knowledge Materials in Python (`PyMKS`) framework is an object-oriented set of tools and examples, written in `Python`, that provide high-level access to the MKS framework for rapid creation and analysis of structure-property-processing relationships. Check the package out if you have not already.

- The __Materials Knowledge Systems (MKS)__ is a novel data science approach for solving multiscale materials science problems. It uses techniques from _physics, machine learning, regression analysis, signal processing, and spatial statistics to create processing-structure-property relationships_. The MKS carries the potential to bridge multiple length scales using localization and homogenization linkages, and provides a data driven framework for solving inverse material design problems.

`H5PY`
- Package to read HDF files

In [16]:
from pymks import PrimitiveBasis
from pymks.stats import correlate
from sklearn.externals import joblib
import numpy as np
import h5py

### **Prediction Function**


    Predicts effective strength of 3-D RVE of a 2-phase composite with strength contrast s2/s1 = 5
    Args: 
    - table_path - path to dream3d file containing microstructure data (phase labels)
    - predictor_path - path to sav file containing calibrated model (LinearRegression)
    - reducer_path - path to sav file containing dimensionality reducer (Principal Component Basis)
    - ms_path - path to microstructure data (phase lables) inside dream3d file
    Returns:
    - y - predicted effective strength 


In [21]:
def predict(table_path,predictor_path,reducer_path,ms_path='/DataContainers/SyntheticVolumeDataContainer/CellData/Phases'):

      
    # Default settings for 2-pt stats
    p_axes = (0,1,2)
    corrs = [(1,1)]
    
    # Read hdf5 table
    f = h5py.File(table_path, 'r')
    data = f[ms_path].value
    ms = np.squeeze(data)
    print "Table loaded successfully..."
    
    # Get phase labels as local states
    states = np.unique(ms)
    if len(states) > 2 :
        print('WARNING: Model is only for two-phase materials! All extra phases will be considered as the second (hard) phase')
        ms[ms > states[0]] = states[0]   
    
    # Get the size of the RVE
    if len(ms.shape) == 4:
        dims = ms.shape[1:4]
    elif len(ms.shape) == 3:
        dims = ms.shape
        ms = np.expand_dims(ms,0)
    else:
        print('ERROR: 3-D RVE(s) are expected!')
        return None
    
    # Load model and dimensionality reducer
    predictor = joblib.load(predictor_path)
    reducer = joblib.load(reducer_path)
    
    # Get the number of PC components used
    n_comps = predictor.named_steps['poly'].n_input_features_

    # Get the size of the calibration RVE
    nx_cal = int(np.round((reducer.components_.shape[1])**(1.0/3.0)))
    dims_cal = np.array((nx_cal,nx_cal,nx_cal))
    
    # Compute 2-pt stats
    n_states = len(states)
    p_basis = PrimitiveBasis(n_states=n_states, domain=states)
    tps = correlate(ms, p_basis, periodic_axes=p_axes, correlations=corrs)
    
    # Check size of the provided MVE: truncate if large, pad if small
    if np.prod(dims) > reducer.components_.shape[1]:
        tps = truncate(tps, [len(ms),dims_cal[0],dims_cal[1],dims_cal[2],1])
        dims = dims_cal
        print('Microstructure volume is larger than calibration RVE. 2-pt correlation function is truncated')
    elif np.prod(dims) < reducer.components_.shape[1]:
        tps = pad(tps, [len(ms),dims_cal[0],dims_cal[1],dims_cal[2],1])
        dims = dims_cal
        print('Microstructure volume is smaller than calibration RVE. 2-pt correlation function is padded')
    
    # Convert 2-pt stats to a vector
    tps_v = np.reshape(tps,(len(ms), np.prod(dims)))

    # Get low-dimensional representation
    x = reducer.transform(tps_v)
    
    # Get the property prediction
    y = predictor.predict(x[:,0:n_comps])
    print "Returning Results... \n"
    return y

def truncate(a, shape):
    '''truncates the edges of the array based on the shape. '''
    
    a_shape = np.array(a.shape)
    n = len(shape)
    new_shape = a_shape.copy()
    new_shape[:n] = shape
    diff_shape = a_shape - new_shape
    index0 = (diff_shape + (diff_shape % 2) * (new_shape % 2)) // 2
    index1 = index0 + new_shape
    multi_slice = tuple(slice(index0[ii], index1[ii]) for ii in range(n))
    return a[multi_slice]

def pad(a, shape):
    '''pads the array with zeros to make for the shape'''
    
    a_shape = np.array(a.shape)
    diff = shape-a_shape
    pad_1 = (diff/2.0).astype(int)
    pad_2 = diff - pad_1
    
    padding = []
    for ii in range(len(pad_1)):
        padding.append((pad_1[ii],pad_2[ii]))
    
    return np.pad(a,padding,'constant',constant_values=(0,0))    

### **Does it Work?**
There's only one way to find out.

In [22]:
predictor_path = 'predictor.sav'
reducer_path = 'reducer.sav'
table_path = 'example_27x27x27.dream3d'


y_hat = predict(table_path,predictor_path,reducer_path)
print "Predicted Strength: ", float(y_hat) 

Table loaded successfully...
Returning Results... 

Predicted Strength:  0.466483628072
